**url :**

https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

In [1]:
#libraries
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd,numpy as np,requests,bs4,re,time,io,pytesseract,easyocr,random,textstat,urllib.request,contextlib
from pdfminer.high_level import extract_text
from PIL import Image
from pathlib import Path
from pdf2image import convert_from_path
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.auto import tqdm

reader = easyocr.Reader(['en'])

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
# option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [3]:
#site name ,link and domain
SITE_NAME='eprocure_bd_gov'

DOMAIN = "https://www.eprocure.gov.bd"

SITE_LINK = "https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t"

In [4]:
#functions
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

def remove_esc_chars(string):
    review = re.sub('''[^a-zA-Z0-9!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~°]''', ' ',string)
    review = review.split()
    review = ' '.join(review)
    return review.strip()

def get_text():
    # View IFT /PQ / REOI / RFP Notice Details
    Ministry  = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[1]/td[2]''').text.strip()
    Division  = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[1]/td[4]''').text.strip()
    Organization = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[2]/td[2]''').text.strip()
    Procuring_Entity_Name = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[2]/td[4]''').text.strip()
    Procuring_Entity_Code = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[3]/td[2]''').text.strip()
    Procuring_Entity_District = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[3]/td[4]''').text.strip()
    Procurement_Nature = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[4]/td[2]''').text.strip()
    Procurement_Type = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[4]/td[4]''').text.strip()
    Event_Type  = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[5]/td[2]''').text.strip()
    Invitation_for = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[5]/td[4]''').text.strip()
    Invitation_Reference_No = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[6]/td[2]''').text.strip()
    Tender_Proposal_Status = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[6]/td[4]''').text.strip()
    App_ID = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[7]/td[2]''').text.strip()
    Tender_Proposal_ID  = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[1]/tbody/tr[7]/td[4]''').text.strip()
    
    #Key Information and Funding Information :
    Procurement_Method =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[2]/tbody/tr[1]/td[2]''').text.strip()
    Budget_Type  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[2]/tbody/tr[1]/td[4]''').text.strip()
    Source_of_Funds  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[2]/tbody/tr[2]/td[2]''').text.strip()
    
    #Particular Information :
    Project_Code  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[1]/td[2]''').text.strip()
    Project_Name =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[1]/td[4]''').text.strip()
    Tender_Proposal_Package_No_and_Description  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[2]/td[2]''').text.strip()
    Category = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[3]/td[2]''').text.strip()
    Scheduled_Tender_Proposal_Publication_Date_and_Time  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[4]/td[2]''').text.strip()
    Tender_Proposal_Document_last_selling_downloading_Date_and_Time =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[4]/td[4]''').text.strip()
    Pre_Tender_Proposal_meeting_Start_Date_and_Time =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[5]/td[2]''').text.strip()
    Pre_Tender_Proposal_meeting_End_Date_and_Time = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[5]/td[4]''').text.strip()
    Tender_Proposal_Closing_Date_and_Time = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[6]/td[2]''').text.strip()
    Tender_Proposal_Opening_Date_and_Time = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[6]/td[4]''').text.strip()
    Last_Date_and_Time_for_Tender_Proposal_Security_Submission = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[3]/tbody/tr[7]/td[2]''').text.strip()
    
    #Information for Tenderer/Consultant :
    Eligibility_of_Tenderer  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[1]/td[2]''').text.strip()
    Brief_Description_of_Works  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[2]/td[2]''').text.strip()
    Evaluation_Type  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[3]/td[2]''').text.strip()
    Document_Available = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[4]/td[2]''').text.strip()
    Document_Fees  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[5]/td[2]''').text.strip()
    Tender_Proposal_Document_Price_In_BDT =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[6]/td[2]''').text.strip()
    Mode_of_Payment =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[7]/td[2]/label''').text.strip()
    Tender_Proposal_Security_Valid_Up_to =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[8]/td[2]''').text.strip()
    Tender_Proposal_Valid_Up_to =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[4]/tbody/tr[9]/td[2]''').text.strip()   
    
    #Procuring Entity Details:
    Name_of_Official_Inviting_Tender_Proposal =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[1]/td[2]''').text.strip()
    Designation_of_Official_Inviting_Tender_Proposal =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[1]/td[4]''').text.strip()
    
    ##Address of Official Inviting Tender/Proposal :
    Address  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td[2]''').text.replace(":","").strip()
    City = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[2]''').text.replace(":","").strip()
    Thana  =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[2]/table/tbody/tr[3]/td[2]''').text.replace(":","").strip()
    District =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[2]/table/tbody/tr[4]/td[2]''').text.replace(":","").strip()
    Country =  driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[2]/table/tbody/tr[5]/td[2]''').text.replace(":","").strip()
    
    ##Contact details of Official Inviting Tender/Proposal :
    Phone_No  = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[4]/table/tbody/tr[1]/td[2]''').text.replace(":","").strip()
    Fax_No = driver.find_element(By.XPATH,f'''//*[@id="frmviewForm"]/table[6]/tbody/tr[2]/td[4]/table/tbody/tr[2]/td[2]''').text.replace(":","").strip()
    
    #url
    url = SITE_LINK
    
    data = (Ministry,Division,Organization,Procuring_Entity_Name,Procuring_Entity_Code,Procuring_Entity_District,
            Procurement_Nature,Procurement_Type,Event_Type,Invitation_for,Invitation_Reference_No,Tender_Proposal_Status,
            App_ID,Tender_Proposal_ID,Procurement_Method,Budget_Type,Source_of_Funds,Project_Code,Project_Name,
            Tender_Proposal_Package_No_and_Description,Category,Scheduled_Tender_Proposal_Publication_Date_and_Time,
            Tender_Proposal_Document_last_selling_downloading_Date_and_Time,Pre_Tender_Proposal_meeting_Start_Date_and_Time,
            Pre_Tender_Proposal_meeting_End_Date_and_Time,Tender_Proposal_Closing_Date_and_Time,
            Tender_Proposal_Opening_Date_and_Time,Last_Date_and_Time_for_Tender_Proposal_Security_Submission,
            Eligibility_of_Tenderer,Brief_Description_of_Works,Evaluation_Type,Document_Available,Document_Fees,
            Tender_Proposal_Document_Price_In_BDT,Mode_of_Payment,Tender_Proposal_Security_Valid_Up_to,Tender_Proposal_Valid_Up_to,
            Name_of_Official_Inviting_Tender_Proposal,Designation_of_Official_Inviting_Tender_Proposal,Address,City,
            Thana,District,Country,Phone_No,Fax_No,url) 
    
    data_cleaned = []
    for x in data:
        x = remove_esc_chars(x)
        data_cleaned.append(x)

    article_list.append(tuple(data_cleaned))
    print(Tender_Proposal_Status,Scheduled_Tender_Proposal_Publication_Date_and_Time,Ministry)
    print(Procuring_Entity_Name,url)
    print()

In [5]:
#get the url
driver.get(SITE_LINK)
time.sleep(3)

In [6]:
#gating data
article_list = []

while True :
    container = driver.find_element(By.XPATH,f'''//*[@id="resultTable"]''')
    elements = container.find_elements(By.XPATH,f'''//*[@id="resultTable"]/tbody/tr''')

    for e in range(1,len(elements[:])+1):
        #getting the link
        link = driver.find_element(By.XPATH,f'''/html/body/div[1]/div/table[2]/tbody/tr/td[2]/div/div[3]/table[1]/tbody/tr[{e}]/td[3]/form/a''').click()
        driver.switch_to.window(driver.window_handles[1])
        get_text()
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    #checking the page number
    pages = driver.find_element(By.XPATH,f'''//*[@id="pagination"]/tbody/tr/td[1]''').text.lower().strip().replace("page","").split("of")
    
    #my condition to break 
    if pages[0].strip() == "2":
        break
        
    #next button click if it is not the last page
    if pages[0].strip() != pages[1].strip():
        driver.find_element(By.XPATH,f'''//*[@id="btnNext"]''').click()
    else:
        print("reached the last page".capitalize())
        break
    time.sleep(3)

Live 28-Jun-2022 10:00 Ministry of Local Government, Rural Development and Co-operatives
Office of the Upazila Engineer, Nagarpur, Tangail https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

Live 28-Jun-2022 10:00 Ministry of Local Government, Rural Development and Co-operatives
Office of the Upazila Engineer, Nagarpur, Tangail https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

Live 28-Jun-2022 10:00 Ministry of Education
Office of The Secretary https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

Live 28-Jun-2022 10:00 Ministry of Local Government, Rural Development and Co-operatives
Bagerhat Paurashava, Dist. Bagerhat https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

Live 28-Jun-2022 10:00 Ministry of Education
Office of The Secretary https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t

Live 28-Jun-2022 10:00 Ministry of Education
Office of the Secretary https://www.eprocure.gov.bd/resources/common/AllTenders.jsp?h=t



In [7]:
#columns for df
columns = ("Ministry","Division","Organization","Procuring_Entity_Name","Procuring_Entity_Code","Procuring_Entity_District",
            "Procurement_Nature","Procurement_Type","Event_Type","Invitation_for","Invitation_Reference_No","Tender_Proposal_Status",
            "App_ID","Tender_Proposal_ID","Procurement_Method","Budget_Type","Source_of_Funds","Project_Code","Project_Name",
            "Tender_Proposal_Package_No_and_Description","Category","Scheduled_Tender_Proposal_Publication_Date_and_Time",
            "Tender_Proposal_Document_last_selling_downloading_Date_and_Time","Pre_Tender_Proposal_meeting_Start_Date_and_Time",
            "Pre_Tender_Proposal_meeting_End_Date_and_Time","Tender_Proposal_Closing_Date_and_Time",
            "Tender_Proposal_Opening_Date_and_Time","Last_Date_and_Time_for_Tender_Proposal_Security_Submission",
            "Eligibility_of_Tenderer","Brief_Description_of_Works","Evaluation_Type","Document_Available","Document_Fees",
            "Tender_Proposal_Document_Price_In_BDT","Mode_of_Payment","Tender_Proposal_Security_Valid_Up_to","Tender_Proposal_Valid_Up_to",
            "Name_of_Official_Inviting_Tender_Proposal","Designation_of_Official_Inviting_Tender_Proposal","Address","City",
            "Thana","District","Country","Phone_No","Fax_No","url") 

In [8]:
#temp df
temp_df = pd.DataFrame(article_list,columns=columns)
temp_df

,Ministry,Division,Organization,Procuring_Entity_Name,Procuring_Entity_Code,Procuring_Entity_District,Procurement_Nature,Procurement_Type,Event_Type,Invitation_for,...,Name_of_Official_Inviting_Tender_Proposal,Designation_of_Official_Inviting_Tender_Proposal,Address,City,Thana,District,Country,Phone_No,Fax_No,url
0,"Ministry of Local Government, Rural Developmen...",Local Government Division,Upazila parishad Nagarpur,"Office of the Upazila Engineer, Nagarpur, Tangail",,Tangail,Goods,NCT,TENDER,Tender - Single Lot,...,Mr. Mahbubur Rahaman,Upazila Engineer,"Upazila Engineer, Nagarpur, Tangail",Nagarpur,Nagarpur,Tangail - 1234,Bangladesh,02-123456,,https://www.eprocure.gov.bd/resources/common/A...
1,"Ministry of Local Government, Rural Developmen...",Local Government Division,Upazila parishad Nagarpur,"Office of the Upazila Engineer, Nagarpur, Tangail",,Tangail,Goods,NCT,TENDER,Tender - Single Lot,...,Mr. Mahbubur Rahaman,Upazila Engineer,"Upazila Engineer, Nagarpur, Tangail",Nagarpur,Nagarpur,Tangail - 1234,Bangladesh,02-123456,,https://www.eprocure.gov.bd/resources/common/A...
2,Ministry of Education,,"Board of Intermediate & Secondary Education, D...",Office of The Secretary,,Dhaka,Goods,NCT,TENDER,Tender - Single Lot,...,Professor Tapan Kumar Sarker,Secretary,"13-14, Joynag Road, Bakshibazar",Dhaka,Chalkbazar,Dhaka - 1211,Bangladesh,02-9669813,,https://www.eprocure.gov.bd/resources/common/A...
3,"Ministry of Local Government, Rural Developmen...",Local Government Division,Bagerhat Paurashava,"Bagerhat Paurashava, Dist. Bagerhat",,Bagerhat,Goods,NCT,TENDER,Tender - Single Lot,...,Md. Wazihur Rahman,Executive Engineer,Bagerhat Paurashava,Bagerhat,Bagerhat Sadar,Bagerhat - 9300,Bangladesh,04-6863108,,https://www.eprocure.gov.bd/resources/common/A...
4,Ministry of Education,,"Board of Intermediate & Secondary Education, D...",Office of The Secretary,,Dhaka,Works,NCT,TENDER,Tender - Single Lot,...,Professor Tapan Kumar Sarker,Secretary,"13-14, Joynag Road, Bakshibazar",Dhaka,Chalkbazar,Dhaka - 1211,Bangladesh,02-9669813,,https://www.eprocure.gov.bd/resources/common/A...
5,Ministry of Education,Secondary and Higher Education Division,"Board of Intermediate and Secondary Education,...",Office of the Secretary,003,Mymensingh,Goods,NCT,TENDER,Tender - Single Lot,...,Professor Kirit Kumar Dutta,Secretary,Govt. Teachers Training College(Male) Thana My...,Mymensingh,Mymensingh,Mymensingh - 2200,Bangladesh,091-63333,,https://www.eprocure.gov.bd/resources/common/A...
6,Ministry of Education,Secondary and Higher Education Division,"Board of Intermediate and Secondary Education,...",Office of the Secretary,003,Mymensingh,Goods,NCT,TENDER,Tender - Single Lot,...,Professor Kirit Kumar Dutta,Secretary,Govt. Teachers Training College(Male) Thana My...,Mymensingh,Mymensingh,Mymensingh - 2200,Bangladesh,091-63333,,https://www.eprocure.gov.bd/resources/common/A...
7,"Ministry of Local Government, Rural Developmen...",Local Government Division,Chittagong City Corporation,Add Chief Engr Office,,Chattogram,Goods,NCT,TENDER,Tender - Single Lot,...,Engr Md Kamrul Islam,Additional Chief Engineer,"Head Office, Tiger Pass, Chattogram",Chattogram,Kotwali,Chattogram - 4000,Bangladesh,031-000000,,https://www.eprocure.gov.bd/resources/common/A...
8,Ministry of Education,Secondary and Higher Education Division,Jessore University of Science & Technology,Office of the Director (P & D),,Jashore,Works,NCT,TENDER,Tender - Single Lot,...,Paritosh Kumar Biswas,Director,Jessore University of Science & Technology,Jessore,Jessore Sadar,Jashore - 7408,Bangladesh,0421-61555,,https://www.eprocure.gov.bd/resources/common/A...
9,"Ministry of Local Government, Rural Developmen...",Local Government Division,Local Government Engineering Department (LGED),"Office of the Upazila Engineer, LGED, Bhedarga...",,Shariatpur,Works,NCT,TENDER,Tender - Single Lot,...,Anupam Chakrabortty,Upazila Engineer,"Bhedarganj Upazila Parishad, Gaidya, Bhedargan...",Bhedarganj,Bhedarganj,Shariatpur -

In [9]:
#to csv
temp_df.to_csv(f'{SITE_NAME} tenders.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} tenders.json')

In [10]:
driver.quit()